In [1]:
import numpy as np
import pandas as pd
import os
import sys
import warnings
warnings.filterwarnings("ignore")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

0 C:\Users\p.schambach\Desktop\DSR\drone_steering\models\archived_notebooks
1 C:\Users\p.schambach\Desktop\DSR\drone_steering\models
2 C:\Users\p.schambach\Desktop\DSR\drone_steering


## Load Data

In [2]:
from app.module import DataEnsembler, GestureTransformer

de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir='data/gesture/',is_frame_based=True)
de.load_data()
de.assemble_data(tolerance_range=2)
de.display_information()

i: 0 	shape X: (603, 18, 16) 	shape y: (603,) 	count: 101
i: 1 	shape X: (519, 18, 16) 	shape y: (519,) 	count: 89
i: 2 	shape X: (498, 18, 16) 	shape y: (498,) 	count: 119
i: 3 	shape X: (650, 18, 16) 	shape y: (650,) 	count: 230
i: 4 	shape X: (508, 18, 16) 	shape y: (508,) 	count: 147
i: 5 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 146
i: 6 	shape X: (601, 18, 16) 	shape y: (601,) 	count: 159
i: 7 	shape X: (440, 18, 16) 	shape y: (440,) 	count: 127
i: 8 	shape X: (490, 18, 16) 	shape y: (490,) 	count: 110
i: 9 	shape X: (476, 18, 16) 	shape y: (476,) 	count: 118
i: 10 	shape X: (492, 18, 16) 	shape y: (492,) 	count: 121
i: 11 	shape X: (436, 18, 16) 	shape y: (436,) 	count: 87
i: 12 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 126
i: 13 	shape X: (551, 18, 16) 	shape y: (551,) 	count: 127
i: 14 	shape X: (550, 18, 16) 	shape y: (550,) 	count: 114
i: 15 	shape X: (549, 18, 16) 	shape y: (549,) 	count: 115
i: 16 	shape X: (402, 18, 16) 	shape y: (402,) 	count: 160
i: 17 	sh

## How to use the DataResampler

In [3]:
from app.module import DataResampler

In [4]:
# Example 1: Take all non-zero-labeled samples and only 500 zero-labeled-samples
dr = DataResampler()
dr.fit(de.X,de.y)
dr.restrict_zero_labels(500)
dr.display_information()

-------------------------------------
Current number of zero-labels: 15777
New number of zero-labels: 500


,abs_orig,pct_orig,abs_current,pct_current
0,15777,75.92,500,9.08
1,701,3.37,701,12.73
2,763,3.67,763,13.86
3,991,4.77,991,18
4,1042,5.01,1042,18.93
5,819,3.94,819,14.88
6,689,3.32,689,12.52


In [5]:
# Example 2: Take all non-zero-labeled samples and only 2000 zero-labeled-samples. Pick the zero-labeled samples according to a certain criterion
dr = DataResampler()
dr.fit(de.X, de.y)
dr.display_information()

crits = ["abs_diff_lowest","abs_diff_highest","variances_lowest","variances_highest"]

for crit in crits:
    dr.restrict_zero_labels(n=2000,criterion=crit)
    print(dr.display_information())

-------------------------------------
Current number of zero-labels: 15777
New number of zero-labels (selected with criterion="abs_diff_lowest"): 2000
  abs_orig pct_orig abs_current pct_current
0    15777    75.92        2000       28.55
1      701     3.37         701       10.01
2      763     3.67         763       10.89
3      991     4.77         991       14.15
4     1042     5.01        1042       14.88
5      819     3.94         819       11.69
6      689     3.32         689        9.84
-------------------------------------
Current number of zero-labels: 15777
New number of zero-labels (selected with criterion="abs_diff_highest"): 2000
  abs_orig pct_orig abs_current pct_current
0    15777    75.92        2000       28.55
1      701     3.37         701       10.01
2      763     3.67         763       10.89
3      991     4.77         991       14.15
4     1042     5.01        1042       14.88
5      819     3.94         819       11.69
6      689     3.32         689      

In [6]:
# Example 3: specify the exact number of samples for each label
a = {1: 200, 3: 150, 4: 400}
dr = DataResampler()
dr.fit(de.X,de.y)
dr.upsample(a)
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,15777,75.92,0,0
1,701,3.37,200,26.67
2,763,3.67,0,0
3,991,4.77,150,20
4,1042,5.01,400,53.33
5,819,3.94,0,0
6,689,3.32,0,0


In [7]:
# Example 4: upsample until there until the size of all groups match the size of the biggest groups (here: the zero-labeld data)
dr = DataResampler()
dr.fit(de.X,de.y)
dr.upsample()
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,15777,75.92,15777,14.29
1,701,3.37,15777,14.29
2,763,3.67,15777,14.29
3,991,4.77,15777,14.29
4,1042,5.01,15777,14.29
5,819,3.94,15777,14.29
6,689,3.32,15777,14.29


In [8]:
# Example 5: Take 1000 of each group. Depending on the initial group size it will upsample or downsample
dr = DataResampler()
dr.fit(de.X,de.y)
dr.upsample(1000)
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,15777,75.92,1000,14.29
1,701,3.37,1000,14.29
2,763,3.67,1000,14.29
3,991,4.77,1000,14.29
4,1042,5.01,1000,14.29
5,819,3.94,1000,14.29
6,689,3.32,1000,14.29


In [9]:
# Example 6: Take 10000 of each group. If the label is in the exceptions list, then keep the corresponding group at initial size
dr = DataResampler()
dr.fit(de.X,de.y)
dr.upsample(10000, exceptions=[0,3])
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,15777,75.92,15777,23.63
1,701,3.37,10000,14.98
2,763,3.67,10000,14.98
3,991,4.77,991,1.48
4,1042,5.01,10000,14.98
5,819,3.94,10000,14.98
6,689,3.32,10000,14.98


In [10]:
# Example 6: Perform a train-test-split
dr = DataResampler()
dr.fit(de.X,de.y)
x_test, y_test = dr.train_test_split(0.2)
# the training data will then be stored in the DataResampler instance
x_train , y_train = dr.X, dr.y
print(dr.display_information())
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(16626, 18, 16) (16626,) (4156, 18, 16) (4156,)


In [16]:
# Example 7: Perform a pseudo train-test-split to shuffle the data
dr = DataResampler()
dr.fit(de.X,de.y)
dr.train_test_split(0.0)
print(dr.display_information())
print("Original:",dr.orig_y[0:30])
print("Suffeled:",dr.y[0:30])

  abs_orig pct_orig abs_current pct_current
0    15777    75.92       15777       75.92
1      701     3.37         701        3.37
2      763     3.67         763        3.67
3      991     4.77         991        4.77
4     1042     5.01        1042        5.01
5      819     3.94         819        3.94
6      689     3.32         689        3.32
Original: [0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
Suffeled: [0 2 0 0 0 6 0 3 0 0 0 0 5 1 0 0 0 0 0 0 0 0 0 0 0 6 3 0 3 0]
